Câu 1. (2.5 điểm) — Cold-start Recommendation

Tính tổng số lượt bình chọn và điểm trung bình của từng bài hát.

Tạo công thức “độ hot”:

score = \text{avg_rating} \times \log(\text{num_votes})

In ra top 5 bài hát nên gợi ý cho user cold-start (người dùng mới chưa có lịch sử nghe nhạc).

👉 Gợi ý: dùng groupby + agg trong Pandas.

In [3]:
#  BÀI 1 – COLD START RECOMMENDATION
import pandas as pd
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity

# ---- Giả lập dữ liệu rating
data = {
    'user_id': [1, 1, 2, 2, 3, 3],
    'song_id': [101, 102, 103, 104, 101, 105],
    'rating': [4, 5, 3, 4, 5, 2]
}
df = pd.DataFrame(data)
print(" Dữ liệu rating:")
print(df, "\n")

# ---- Tính số lượt bình chọn và điểm trung bình
baiHat = df.groupby('song_id').agg(
    soLuotBinhChon=('rating', 'count'),
    soDiemTrungBinh=('rating', 'mean')
)
baiHat['score'] = baiHat['soDiemTrungBinh'] * np.log(baiHat['soLuotBinhChon'])

# ---- Top 5 bài hát nên nghe (user cold-start)
top5 = baiHat.sort_values('score', ascending=False).head(5)
print("🎧 Top 5 bài hát nên nghe cho user cold-start:")
print(top5, "\n")

#  BÀI 2 – CONTENT-BASED RECOMMENDATION

songs = {
    'song_id': [101, 102, 103, 104, 105],
    'title': ["Love Again", "Thunder", "Skyline", "Deep Ocean", "Blaze Up"],
    'genres': ["Pop|Ballad", "Rock|Pop", "Indie|Alternative", "Ambient|Chill", "Rock|Metal"]
}
df_songs = pd.DataFrame(songs)
print(" Dữ liệu bài hát:")
print(df_songs, "\n")

# ---- Chuyển thể loại sang set
df_songs['genres'] = df_songs['genres'].apply(lambda x: set(x.split('|')))

# ---- Chọn phim đầu tiên làm ví dụ
phim1 = df_songs.iloc[0]

tap_tuongDong = []
for i, phim in df_songs.iterrows():
    if phim['song_id'] != phim1['song_id']:
        giao = len(phim1['genres'].intersection(phim['genres']))
        hop = len(phim1['genres'].union(phim['genres']))
        tuongDong = giao / hop if hop > 0 else 0
        tap_tuongDong.append({
            'song_id': phim['song_id'],
            'title': phim['title'],
            'genres': phim['genres'],
            'tuongDong': tuongDong
        })

doTuongDong = pd.DataFrame(tap_tuongDong)
doTuongDong_top = doTuongDong.sort_values('tuongDong', ascending=False).head(10)
print(f"🎥 Top 10 bài hát tương đồng với '{phim1['title']}':")
print(doTuongDong_top, "\n")
# 🔹 BÀI 3 – COLLABORATIVE FILTERING
# ---- Tạo ma trận user–item
utility_matrix = df.pivot_table(index='user_id', columns='song_id', values='rating')
print("📊 Ma trận User–Item:")
print(utility_matrix, "\n")

# ---- Tính độ tương đồng giữa các user
user_similarity = pd.DataFrame(
    cosine_similarity(utility_matrix.fillna(0)),
    index=utility_matrix.index,
    columns=utility_matrix.index
)
print(" Ma trận tương đồng giữa các User:")
print(user_similarity, "\n")

# ---- Tính độ tương đồng giữa các item
item_similarity = pd.DataFrame(
    cosine_similarity(utility_matrix.fillna(0).T),
    index=utility_matrix.columns,
    columns=utility_matrix.columns
)
print(" Ma trận tương đồng giữa các Item:")
print(item_similarity, "\n")

# ---- Gợi ý bài hát cho từng user dựa trên user tương tự
print(" Gợi ý bài hát cho từng user:")
for user in utility_matrix.index:
    listened = utility_matrix.loc[user][utility_matrix.loc[user].notna()].index
    not_listened = utility_matrix.columns.difference(listened)

    similar_users = user_similarity[user].drop(user)
    weights = similar_users / similar_users.sum()

    pred_ratings = {}
    for song in not_listened:
        # 🔧 CHỈ LẤY rating CỦA CÁC USER TRONG weights.index
        other_ratings = utility_matrix.loc[weights.index, song]
        pred = np.dot(weights, other_ratings.fillna(0))
        pred_ratings[song] = pred

    top_items = sorted(pred_ratings.items(), key=lambda x: x[1], reverse=True)[:2]
    print(f"- User {user} nên nghe:", [i[0] for i in top_items])

print("\n Hoàn thành 3 phần: Cold-start, Content-based, Collaborative Filtering!")


 Dữ liệu rating:
   user_id  song_id  rating
0        1      101       4
1        1      102       5
2        2      103       3
3        2      104       4
4        3      101       5
5        3      105       2 

🎧 Top 5 bài hát nên nghe cho user cold-start:
         soLuotBinhChon  soDiemTrungBinh     score
song_id                                           
101                   2              4.5  3.119162
102                   1              5.0  0.000000
103                   1              3.0  0.000000
104                   1              4.0  0.000000
105                   1              2.0  0.000000 

 Dữ liệu bài hát:
   song_id       title             genres
0      101  Love Again         Pop|Ballad
1      102     Thunder           Rock|Pop
2      103     Skyline  Indie|Alternative
3      104  Deep Ocean      Ambient|Chill
4      105    Blaze Up         Rock|Metal 

🎥 Top 10 bài hát tương đồng với 'Love Again':
   song_id       title                genres  tuongDong
0     

Cold-start (top bài hát/phim nên gợi ý cho user mới)

💡 Dùng file ratings.csv trong MovieLens.

Gợi ý đề thi:

Từ dữ liệu rating, hãy đề xuất top 5 movies nên xem cho user mới (chưa có lịch sử).
Gợi ý: kết hợp số lượt rating và rating trung bình, tạo công thức:

𝑠
𝑐
𝑜
𝑟
𝑒
=
𝑚
𝑒
𝑎
𝑛
_
𝑟
𝑎
𝑡
𝑖
𝑛
𝑔
×
log
⁡
(
𝑛
𝑢
𝑚
_
𝑟
𝑎
𝑡
𝑖
𝑛
𝑔
𝑠
)
score=mean_rating×log(num_ratings)

In [5]:
import pandas as pd
import numpy as np

df = pd.read_csv('movie_lens/ratings.csv')

movie_stats = df.groupby('movieId').agg(
    num_ratings=('rating', 'count'),
    avg_rating=('rating', 'mean')
)
movie_stats['score'] = movie_stats['avg_rating'] * np.log(movie_stats['num_ratings'])

top5 = movie_stats.sort_values('score', ascending=False).head(5)
print("Top 5 phim nên gợi ý cho user cold-start:")
print(top5)


Top 5 phim nên gợi ý cho user cold-start:
         num_ratings  avg_rating      score
movieId                                    
318              317    4.429022  25.506303
356              329    4.164134  24.135560
296              307    4.197068  24.035972
2571             278    4.192446  23.593498
593              279    4.161290  23.433107


Content-Based Recommendation (theo thể loại phim)

💡 Dùng file movies.csv, trong đó có cột genres.
Thầy thường yêu cầu tính độ tương đồng Jaccard giữa 2 phim theo thể loại.

Gợi ý đề thi:

Từ file movies.csv, chọn một phim X và tính top 10 phim tương đồng nhất với X theo thể loại bằng độ đo Jaccard

In [6]:
movies = pd.read_csv('movie_lens/movies.csv')
movies['genres'] = movies['genres'].apply(lambda x: set(x.split('|')))

film_x = movies.iloc[0]  # chọn phim đầu tiên

similarity_list = []
for i, film in movies.iterrows():
    if film['movieId'] != film_x['movieId']:
        inter = len(film_x['genres'].intersection(film['genres']))
        union = len(film_x['genres'].union(film['genres']))
        jaccard = inter / union if union > 0 else 0
        similarity_list.append({
            'movieId': film['movieId'],
            'title': film['title'],
            'similarity': jaccard
        })

result = pd.DataFrame(similarity_list).sort_values('similarity', ascending=False)
print(f"Top 10 phim tương tự '{film_x['title']}':")
print(result.head(10))


Top 10 phim tương tự 'Toy Story (1995)':
      movieId                                              title  similarity
2808     3754     Adventures of Rocky and Bullwinkle, The (2000)         1.0
9429   166461                                       Moana (2016)         1.0
7759    91355  Asterix and the Vikings (Astérix et les Viking...         1.0
6947    65577                     Tale of Despereaux, The (2008)         1.0
6485    53121                             Shrek the Third (2007)         1.0
3567     4886                              Monsters, Inc. (2001)         1.0
1705     2294                                        Antz (1998)         1.0
8218   103755                                       Turbo (2013)         1.0
2999     4016                   Emperor's New Groove, The (2000)         1.0
8926   136016                           The Good Dinosaur (2015)         1.0


Collaborative Filtering (lọc cộng tác)

💡 Dùng file ratings.csv → pivot thành ma trận User × Movie.
Sau đó tính độ tương đồng cosine giữa user hoặc giữa item.

Gợi ý đề thi:

Tính ma trận tương đồng giữa các user (user-based CF) và giữa các item (item-based CF).
Với mỗi user, gợi ý 2 phim họ chưa xem dựa trên user tương tự.

In [8]:
from sklearn.metrics.pairwise import cosine_similarity

ratings = pd.read_csv('movie_lens/ratings.csv')
pivot = ratings.pivot_table(index='userId', columns='movieId', values='rating')

# Độ tương đồng giữa user
user_sim = pd.DataFrame(
    cosine_similarity(pivot.fillna(0)),
    index=pivot.index, columns=pivot.index
)

# Độ tương đồng giữa item
item_sim = pd.DataFrame(
    cosine_similarity(pivot.fillna(0).T),
    index=pivot.columns, columns=pivot.columns
)

print("User similarity matrix:")
print(user_sim.head(3))

print("Item similarity matrix:")
print(item_sim.head(3))


User similarity matrix:
userId       1         2        3         4         5         6         7    \
userId                                                                        
1       1.000000  0.027283  0.05972  0.194395  0.129080  0.128152  0.158744   
2       0.027283  1.000000  0.00000  0.003726  0.016614  0.025333  0.027585   
3       0.059720  0.000000  1.00000  0.002251  0.005020  0.003936  0.000000   

userId       8         9         10   ...       601       602       603  \
userId                                ...                                 
1       0.136968  0.064263  0.016875  ...  0.080554  0.164455  0.221486   
2       0.027257  0.000000  0.067445  ...  0.202671  0.016866  0.011997   
3       0.004941  0.000000  0.000000  ...  0.005048  0.004892  0.024992   

userId       604       605       606       607       608       609       610  
userId                                                                        
1       0.070669  0.153625  0.164191  0.269389

Có thể thêm (dự đoán missing rating)

Thầy có thể hỏi kiểu:

“Dự đoán giá trị rating còn thiếu bằng Pearson / Cosine similarity”.

📘 Gợi ý:

Với Pearson, chuẩn hóa mỗi hàng (trừ đi trung bình từng user).

Với Cosine, chỉ cần điền NaN = 0 rồi tính bằng cosine_similarity.

In [1]:
# 1. User-based Collaborative Filtering

#Ý tưởng: dự đoán rating của user dựa trên các user tương tự.
import numpy as np
import pandas as pd

# Ma trận user-item
ratings = pd.DataFrame({
    'I1': [4, 5, 2, np.nan],
    'I2': [3, 2, 5, 3],
    'I3': [np.nan, 2, 4, 3],
    'I4': [4, 5, 1, 2]
}, index=['U1', 'U2', 'U3', 'U4'])

# Tính cosine similarity giữa users
from sklearn.metrics.pairwise import cosine_similarity
user_sim = pd.DataFrame(cosine_similarity(ratings.fillna(0)),
                        index=ratings.index, columns=ratings.index)

# Dự đoán rating thiếu (ví dụ U1, I3)
target_user, target_item = 'U1', 'I3'
neighbors = user_sim[target_user].drop(target_user)

# Lấy user có rating cho I3
valid_users = ratings[~ratings[target_item].isna()].index
numerator, denominator = 0, 0
mean_u = ratings.loc[target_user].mean()

for v in valid_users:
    sim = user_sim.loc[target_user, v]
    mean_v = ratings.loc[v].mean()
    numerator += sim * (ratings.loc[v, target_item] - mean_v)
    denominator += abs(sim)

pred = mean_u + numerator / denominator
print(f"Dự đoán rating {target_user}-{target_item}: {pred:.2f}")


Dự đoán rating U1-I3: 3.38


In [3]:
#tem-based Collaborative Filtering

#Ý tưởng: dự đoán rating của user cho item dựa vào các item tương tự mà user đã đánh giá.
item_sim = pd.DataFrame(cosine_similarity(ratings.fillna(0).T),
                        index=ratings.columns, columns=ratings.columns)

target_user, target_item = 'U1', 'I3'
rated_items = ratings.loc[target_user].dropna()

numerator, denominator = 0, 0
for j in rated_items.index:
    sim = item_sim.loc[target_item, j]
    numerator += sim * ratings.loc[target_user, j]
    denominator += abs(sim)

pred = numerator / denominator
print(f"Dự đoán rating {target_user}-{target_item}: {pred:.2f}")


Dự đoán rating U1-I3: 3.54


In [4]:
#tính jaccard 
from sklearn.metrics import jaccard_score

item1 = [1, 0, 1, 1, 0]
item2 = [1, 1, 0, 1, 0]
sim = jaccard_score(item1, item2)
print("Độ tương đồng Jaccard:", sim)

Độ tương đồng Jaccard: 0.5


In [5]:
#Content-based bằng TF-IDF (văn bản mô tả)
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

docs = [
    "AI machine learning data",
    "deep learning neural network",
    "AI data science project",
    "web development html css"
]

vectorizer = TfidfVectorizer()
tfidf = vectorizer.fit_transform(docs)
sim = cosine_similarity(tfidf)

print("Ma trận tương đồng giữa các item:\n", sim)


Ma trận tương đồng giữa các item:
 [[1.         0.19297924 0.4078538  0.        ]
 [0.19297924 1.         0.         0.        ]
 [0.4078538  0.         1.         0.        ]
 [0.         0.         0.         1.        ]]


In [6]:
#Dự đoán rating bằng SVD (dạng cơ bản)

from sklearn.decomposition import TruncatedSVD

R = ratings.fillna(0).values
svd = TruncatedSVD(n_components=2)
U = svd.fit_transform(R)
V = svd.components_

pred = np.dot(U, V)
pred_df = pd.DataFrame(pred, index=ratings.index, columns=ratings.columns)
print(pred_df.round(2))


      I1    I2    I3    I4
U1  4.14  2.15  0.89  4.11
U2  4.90  2.73  1.23  4.88
U3  1.41  4.81  4.16  1.66
U4  0.87  3.23  2.82  1.04


In [7]:
#🧩 6. Dự đoán missing value (bài cold-start nhỏ)
import numpy as np
import pandas as pd

data = pd.DataFrame({
    'I1': [4, 3, 2],
    'I2': [3, 1, 3],
    'I3': [np.nan, 2, 3],
    'I4': [4, 3, 2]
}, index=['U1', 'U2', 'U3'])

from sklearn.metrics.pairwise import cosine_similarity
sim = pd.DataFrame(cosine_similarity(data.fillna(0)),
                   index=data.index, columns=data.index)

target_user, target_item = 'U1', 'I3'
mean_u = data.loc[target_user].mean()
numerator, denominator = 0, 0

for v in data.index:
    if v != target_user and not np.isnan(data.loc[v, target_item]):
        sim_uv = sim.loc[target_user, v]
        mean_v = data.loc[v].mean()
        numerator += sim_uv * (data.loc[v, target_item] - mean_v)
        denominator += abs(sim_uv)

pred = mean_u + numerator / denominator
print("Dự đoán rating:", round(pred, 2))


Dự đoán rating: 3.77


In [ ]:
# ============================================
# BÀI 3: LỌC CỘNG TÁC (COLLABORATIVE FILTERING)
# Sử dụng kNN (user-based & item-based)
# ============================================

import pandas as pd
from sklearn.neighbors import NearestNeighbors
import numpy as np

# Đọc dữ liệu
df = pd.read_csv('data.csv', names=['userId', 'songId', 'rating'])
df = df.dropna(subset=['userId', 'songId', 'rating'])
df['rating'] = pd.to_numeric(df['rating'], errors='coerce')
df = df.groupby(['userId', 'songId'])['rating'].mean().reset_index()

# -------------------------------
# Tạo ma trận user-item
# -------------------------------
pivot = df.pivot_table(index='userId', columns='songId', values='rating').fill_value=0
pivot = df.pivot_table(index='userId', columns='songId', values='rating').fillna(0)

print("📊 Ma trận user-item:")
display(pivot.head())

# ============================================
# 1️⃣ USER-BASED COLLABORATIVE FILTERING (kNN)
# ============================================

print("\n=== USER-BASED CF ===")

# Mô hình kNN
model_user = NearestNeighbors(metric='cosine', algorithm='brute')
model_user.fit(pivot)

# Chọn 1 user bất kỳ (ví dụ user đầu tiên)
target_user = pivot.index[0]
print(f"🎯 Gợi ý cho user: {target_user}")

# Tìm k user tương tự nhất
distances, indices = model_user.kneighbors(pivot.loc[[target_user]], n_neighbors=6)

# In ra top 5 user tương tự
print("\n👥 Top 5 user tương tự:")
for i in range(1, 6):
    print(f"{i}. User: {pivot.index[indices.flatten()[i]]}, Similarity: {1 - distances.flatten()[i]:.3f}")

# -------------------------------
# Gợi ý bài hát cho user
# -------------------------------
# Tính trung bình rating của các user tương tự để đề xuất bài chưa nghe
similar_users = pivot.index[indices.flatten()[1:6]]
mean_ratings = pivot.loc[similar_users].mean().sort_values(ascending=False)
recommended_songs_user = mean_ratings.head(5)

print("\n🎵 Gợi ý bài hát (theo user-based CF):")
print(recommended_songs_user)

# ============================================
# 2️⃣ ITEM-BASED COLLABORATIVE FILTERING (kNN)
# ============================================

print("\n=== ITEM-BASED CF ===")

# Chuyển vị để các cột là user, hàng là bài hát
pivot_item = pivot.T

# Mô hình kNN
model_item = NearestNeighbors(metric='cosine', algorithm='brute')
model_item.fit(pivot_item)

# Chọn 1 bài hát bất kỳ (ví dụ bài đầu tiên)
target_song = pivot_item.index[0]
print(f"🎯 Gợi ý bài hát tương tự với: {target_song}")

# Tìm k bài hát tương tự
distances_item, indices_item = model_item.kneighbors(pivot_item.loc[[target_song]], n_neighbors=6)

print("\n🎧 Top 5 bài hát tương tự (item-based):")
for i in range(1, 6):
    print(f"{i}. Bài hát: {pivot_item.index[indices_item.flatten()[i]]}, Similarity: {1 - distances_item.flatten()[i]:.3f}")


In [8]:
# ====== IMPORTS ======
import pandas as pd
import numpy as np
from sklearn.neighbors import NearestNeighbors

# ====== 0. LOAD DATA (thay đường dẫn nếu cần) ======
# Hỗ trợ cả 2 dạng: ('user','title','rating') hoặc ('userId','songId','rating')
df = pd.read_csv('data.csv', names=['user', 'title', 'rating'])  # nếu file đã có header, bỏ names=
# Nếu file của bạn có header 'userId','songId','rating', dùng:
# df = pd.read_csv('data.csv')

# Nếu cột tên khác, normalize tên cột để thống nhất
cols = [c.lower().strip() for c in df.columns]
rename_map = {}
if 'user' in cols:
    rename_map[df.columns[cols.index('user')]] = 'user'
elif 'userid' in cols:
    rename_map[df.columns[cols.index('userid')]] = 'user'
if 'title' in cols:
    rename_map[df.columns[cols.index('title')]] = 'item'
elif 'songid' in cols:
    rename_map[df.columns[cols.index('songid')]] = 'item'
if 'rating' in cols:
    rename_map[df.columns[cols.index('rating')]] = 'rating'

df = df.rename(columns=rename_map)[['user','item','rating']]

# ====== 1. TIỀN XỬ LÝ (CLEANING) ======
# 1.1 drop NA
df = df.dropna(subset=['user','item','rating'])

# 1.2 ép kiểu, strip
df['user'] = df['user'].astype(str).str.strip()
df['item'] = df['item'].astype(str).str.strip()
df['rating'] = pd.to_numeric(df['rating'], errors='coerce')

# 1.3 loại rating bất thường (nếu biết scale, ví dụ 1-5)
df = df[(df['rating'] >= 1) & (df['rating'] <= 5)]

# 1.4 nếu một user rate 1 item nhiều lần, lấy trung bình (hoặc latest tuỳ ý)
df = df.groupby(['user','item'])['rating'].mean().reset_index()

print("Data after cleaning:", df.shape)
print(df.head())

# ====== 2. PIVOT: tạo user-item matrix ======
# Mỗi hàng = 1 user, mỗi cột = 1 item
user_item = df.pivot(index='user', columns='item', values='rating')

# Thông thường điền NaN bằng 0 cho thuật toán kNN cosine; 
# nhưng chú ý: 0 có thể mang ý nghĩa "không đánh giá" — tùy chọn khác là fill bằng user mean khi cần
user_item_fill0 = user_item.fillna(0)

print("User-Item matrix shape:", user_item_fill0.shape)
display(user_item_fill0.head())

# ====== 3. OPTION: MEAN-CENTERING (khử bias người dùng) ======
# Khi dùng cosine, nhiều tài liệu khuyến khích mean-centering (rating - user_mean)
user_means = user_item.mean(axis=1)                      # mean trên hàng (user)
user_item_centered = user_item.sub(user_means, axis=0)   # NaN preserved
user_item_centered_filled = user_item_centered.fillna(0) # điền 0 cho phần chưa rate (tức là trung hòa)

# Bạn có thể dùng user_item_fill0 (raw) hoặc user_item_centered_filled (centered)
MATRIX = user_item_centered_filled   # chọn ma trận để train kNN

# ====== 4. USER-BASED kNN: tìm người tương tự & gợi ý ======
def user_based_recommend(user_id, matrix=MATRIX, df_original=df, k=5, n_recommendations=10):
    """
    Trả về top-n gợi ý cho user_id dựa trên user-based kNN.
    - matrix: DataFrame user x item (đã fillna)
    - df_original: original ratings dataframe để lọc items đã có của user
    """
    if user_id not in matrix.index:
        raise ValueError("User không tồn tại trong dữ liệu.")

    model = NearestNeighbors(metric='cosine', algorithm='brute')
    model.fit(matrix.values)

    user_vec = matrix.loc[[user_id]].values
    distances, indices = model.kneighbors(user_vec, n_neighbors=k+1)  # +1 vì chính user sẽ là neighbor thứ 0

    similar_user_idxs = indices.flatten()[1:]     # bỏ chính user
    similar_users = matrix.index[similar_user_idxs]
    sims = 1 - distances.flatten()[1:]            # similarity = 1 - cosine_distance

    # Lấy các item mà similar users đánh giá, tính weighted score
    sim_df = pd.DataFrame({'user': similar_users, 'sim': sims})
    # merge để có ratings
    merged = sim_df.merge(df_original, on='user')   # (user, sim) + (user,item,rating)
    # loại bỏ item user đã có
    items_already = set(df_original[df_original['user']==user_id]['item'])
    candidates = merged[~merged['item'].isin(items_already)].copy()

    # weighted score: sim * rating, rồi sum / sum(sim)
    candidates['weighted'] = candidates['sim'] * candidates['rating']
    score = candidates.groupby('item').agg(weighted_sum=('weighted','sum'), sim_sum=('sim','sum'))
    score['score'] = score['weighted_sum'] / score['sim_sum']
    score = score.sort_values('score', ascending=False).head(n_recommendations)
    return score.reset_index()

# Ví dụ: gợi ý cho user đầu tiên
example_user = MATRIX.index[0]
rec_for_user = user_based_recommend(example_user, k=5, n_recommendations=10)
print(f"\nRecommendations for user {example_user}:")
print(rec_for_user)

# ====== 5. ITEM-BASED kNN: tìm item tương tự ======
def item_based_recommend(item_id, matrix=MATRIX, df_original=df, k=10, n_neighbors=5):
    """
    Trả về top-n item tương tự cho item_id (item-based kNN).
    - matrix: user x item (centered and filled) -> transpose inside
    """
    if item_id not in matrix.columns:
        raise ValueError("Item không tồn tại trong dữ liệu.")

    item_matrix = matrix.T  # now rows = items, cols = users
    model = NearestNeighbors(metric='cosine', algorithm='brute')
    model.fit(item_matrix.values)

    item_vec = item_matrix.loc[[item_id]].values
    distances, indices = model.kneighbors(item_vec, n_neighbors=k+1)

    neighbor_idxs = indices.flatten()[1:]
    neighbors = item_matrix.index[neighbor_idxs]
    sims = 1 - distances.flatten()[1:]

    out = pd.DataFrame({"item": neighbors, "similarity": sims})
    return out.head(n_neighbors)

# Ví dụ: tương tự với item đầu tiên
example_item = MATRIX.columns[0]
print(f"\nItems similar to {example_item}:")
print(item_based_recommend(example_item, k=10, n_neighbors=5))

# ====== 6. LƯU & NOTE ======
# Lưu ma trận (nếu cần)
user_item_fill0.to_csv('user_item_fill0.csv')
user_item_centered_filled.to_csv('user_item_centered.csv')
print("\nSaved pivot matrices to CSV.")


Data after cleaning: (400, 3)
  user item  rating
0    1   18     5.0
1   10   13     4.0
2   10   14     5.0
3   10   15     2.0
4   10   16     2.0
User-Item matrix shape: (81, 20)


item,1,10,11,12,13,14,15,16,17,18,19,2,20,3,4,5,6,7,8,9
user,,,,,,,,,,,,,,,,,,,,
1,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,5.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
10,0.0,0.0,0.0,0.0,4.0,5.000000,2.000000,2.0,0.0,0.0,3.0,2.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,5.0
100,0.0,0.0,3.0,0.0,0.0,0.000000,5.000000,2.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5.0,0.0,0.0
11,0.0,0.0,0.0,3.5,2.0,0.000000,1.000000,4.5,4.0,0.0,0.0,0.0,0.0,5.0,2.0,3.0,3.0,3.0,0.0,3.5
12,0.0,3.0,0.0,1.0,0.0,2.333333,3.333333,0.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0,1.0,2.0,0.0,1.0,0.0



Recommendations for user 1:
  item  weighted_sum  sim_sum  score
0    1           0.0      0.0    NaN
1   10           0.0      0.0    NaN
2   11           0.0      0.0    NaN
3   12           0.0      0.0    NaN
4   13           0.0      0.0    NaN
5   14           0.0      0.0    NaN
6   15           0.0      0.0    NaN
7   16           0.0      0.0    NaN
8   17           0.0      0.0    NaN
9   19           0.0      0.0    NaN

Items similar to 1:
  item  similarity
0   20    0.260248
1    9    0.181275
2   14    0.032383
3   17    0.026995
4   18    0.025686

Saved pivot matrices to CSV.


huật toán lọc cộng tác (Collaborative Filtering) là một kỹ thuật gợi ý dựa trên sự tương đồng giữa người dùng hoặc giữa các sản phẩm. Trong bài, chúng em sử dụng phương pháp k-Nearest Neighbors (kNN) để tìm các người dùng hoặc bài hát có hành vi tương tự và từ đó đề xuất những bài hát mà người dùng có thể yêu thích.”